# Flipside Crypto SDK
- Author: Steffan Rees
- 29/08/2022

Explore using Flipside's SDK in Python.

## Resources
- https://github.com/FlipsideCrypto/sdk
- https://docs.flipsidecrypto.com/shroomdk-sdk/getting-started
- https://sdk.flipsidecrypto.xyz/shroomdk
- https://app.flipsidecrypto.com/velocity
- https://teamflipside.notion.site/teamflipside/SQL-Basics-8b3a10aa657f4b84bc80229f4619b0ea
- https://www.youtube.com/watch?v=VB0ZlZ_Hr_A
- https://github.com/FlipsideCrypto/sdk/tree/main/examples/python

In [1]:
from datetime import datetime
import math
import toml
from shroomdk import ShroomDK

In [2]:
# Read config file and import configurations
config = toml.load('../config.toml')
SHROOMDK_API = config.get('shroomdk').get('shroomdk_api')

In [3]:
# Initialise ShroomDK
sdk = ShroomDK(SHROOMDK_API)

In [4]:
# Constants
# The max number of rows a query can return and cache is 1M.
# However, only 100k rows/records can be returned in a single http call. 
MAX_QUERY_LIMIT = 1000000
PAGE_SIZE = 100000

### Example: get all Solana swaps in the last 15 days
https://www.youtube.com/watch?v=bD1ioDw41aw

In [5]:
# First need to get the total number of swaps and start and end dates
query_count = """
SELECT
  COUNT(1) as swap_count,
  GETDATE() as end_time,
  GETDATE() - interval '15 days' as start_time
FROM
  solana.fact_swaps
WHERE
  succeeded = true
  AND block_timestamp BETWEEN GETDATE() - interval '15 days'
  AND GETDATE()
"""

In [6]:
qrs_count = sdk.query(query_count)
qrs_count

QueryResultSet(query_id=None, status='finished', columns=['SWAP_COUNT', 'END_TIME', 'START_TIME'], column_types=['number', 'timestamp', 'timestamp'], rows=[[1006506, '2022-08-30 10:02:17.242 +0000', '2022-08-15 10:02:17.242 +0000']], run_stats=QueryRunStats(started_at=datetime.datetime(2022, 8, 30, 10, 2, 17), ended_at=datetime.datetime(2022, 8, 30, 10, 2, 19), elapsed_seconds=2, record_count=1), records=[{'swap_count': 1006506, 'end_time': '2022-08-30 10:02:17.242 +0000', 'start_time': '2022-08-15 10:02:17.242 +0000'}], error=None)

In [7]:
# Total number of Solana swaps over the last 15 days
count = qrs_count.records[0]["swap_count"]
count

1006506

In [8]:
start_time = qrs_count.records[0]["start_time"]
start_time

'2022-08-15 10:02:17.242 +0000'

In [9]:
end_time = qrs_count.records[0]["end_time"]
end_time

'2022-08-30 10:02:17.242 +0000'

In [10]:
query_runs = math.ceil(count/MAX_QUERY_LIMIT)
query_runs

2

In [11]:
all_records = []
start_recorded_time = datetime.now()
for query_run in range(query_runs):
    offset = query_run * MAX_QUERY_LIMIT
    query = f"""
    SELECT
      *
    FROM
      solana.fact_swaps
    WHERE
      succeeded = true
      AND block_timestamp BETWEEN '{start_time}' AND '{end_time}'
    ORDER BY
      block_timestamp DESC
    LIMIT
      {MAX_QUERY_LIMIT}
    OFFSET
      {offset}
    """
    for page_number in range(1, int(round(MAX_QUERY_LIMIT/PAGE_SIZE))+1):
        print(f"Querying {PAGE_SIZE} records from offset {offset} @ page {page_number}")
        qrs = sdk.query(query, page_number=page_number, page_size=PAGE_SIZE, ttl_minutes=1)
        if qrs.records is None:
            continue
        all_records.extend(qrs.records)
        print(f"Records returned thus far: {len(all_records)}")
        if len(all_records) == count:
            break
    
print(f"Query took {datetime.now() - start_recorded_time} in total to run")
print(f"Total number of records returned: {len(all_records)}")

Querying 100000 records from offset 0 @ page 1
Records returned thus far: 100000
Querying 100000 records from offset 0 @ page 2
Records returned thus far: 200000
Querying 100000 records from offset 0 @ page 3
Records returned thus far: 300000
Querying 100000 records from offset 0 @ page 4
Records returned thus far: 400000
Querying 100000 records from offset 0 @ page 5
Records returned thus far: 500000
Querying 100000 records from offset 0 @ page 6
Records returned thus far: 600000
Querying 100000 records from offset 0 @ page 7
Records returned thus far: 700000
Querying 100000 records from offset 0 @ page 8
Records returned thus far: 800000
Querying 100000 records from offset 0 @ page 9
Records returned thus far: 900000
Querying 100000 records from offset 0 @ page 10
Records returned thus far: 1000000
Querying 100000 records from offset 1000000 @ page 1
Records returned thus far: 1006506
Query took 0:03:39.484729 in total to run
Total number of records returned: 1006506


In [12]:
all_records[0]

{'swap_program': 'jupiter aggregator v2',
 'block_timestamp': '2022-08-30 07:11:56.000',
 'block_id': 148315779,
 'tx_id': 'maXy87h2CBq78qxBTTnG8kPLN6pXi18BuWrDLuC6BtKn9FgjznTWXHhnF68mSv6Qu8qqDNdVs3c8JW1sXKrqK1p',
 'succeeded': True,
 'swapper': 'Dd1k91cWt84qJoQr3FT7EXQpSaMtZtwPwdho7RbMWtEV',
 'swap_from_amount': 32.814999,
 'swap_from_mint': 'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB',
 'swap_to_amount': 32.814667,
 'swap_to_mint': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v'}

## Adapted example: get all UniswapV2 swaps in the last 15 days

In [13]:
# First need to get the total number of swaps and start and end dates
query_count = """
SELECT
  COUNT(1) as swap_count,
  GETDATE() as end_time,
  GETDATE() - interval '15 days' as start_time
FROM
  ethereum.core.ez_dex_swaps
WHERE
  platform = 'uniswap-v2'
  AND event_name = 'Swap'
  AND block_timestamp BETWEEN GETDATE() - interval '15 days'
  AND GETDATE()
"""

In [14]:
qrs_count = sdk.query(query_count)
qrs_count

QueryResultSet(query_id=None, status='finished', columns=['SWAP_COUNT', 'END_TIME', 'START_TIME'], column_types=['number', 'timestamp', 'timestamp'], rows=[[1193027, '2022-08-30 10:05:42.148 +0000', '2022-08-15 10:05:42.148 +0000']], run_stats=QueryRunStats(started_at=datetime.datetime(2022, 8, 30, 10, 5, 42), ended_at=datetime.datetime(2022, 8, 30, 10, 5, 42), elapsed_seconds=0, record_count=1), records=[{'swap_count': 1193027, 'end_time': '2022-08-30 10:05:42.148 +0000', 'start_time': '2022-08-15 10:05:42.148 +0000'}], error=None)

In [15]:
# Total number of UniswapV2 swaps over the last 15 days
count = qrs_count.records[0]["swap_count"]
count

1193027

In [16]:
start_time = qrs_count.records[0]["start_time"]
start_time

'2022-08-15 10:05:42.148 +0000'

In [17]:
end_time = qrs_count.records[0]["end_time"]
end_time

'2022-08-30 10:05:42.148 +0000'

In [18]:
query_runs = math.ceil(count/MAX_QUERY_LIMIT)
query_runs

2

In [19]:
all_records = []
start_recorded_time = datetime.now()
for query_run in range(query_runs):
    offset = query_run * MAX_QUERY_LIMIT
    query = f"""
    SELECT
      *
    FROM
      ethereum.core.ez_dex_swaps
    WHERE
      platform = 'uniswap-v2'
      AND event_name = 'Swap'
      AND block_timestamp BETWEEN '{start_time}' AND '{end_time}'
    ORDER BY
      block_timestamp DESC
    LIMIT
      {MAX_QUERY_LIMIT}
    OFFSET
      {offset}
    """
    for page_number in range(1, int(round(MAX_QUERY_LIMIT/PAGE_SIZE))+1):
        print(f"Querying {PAGE_SIZE} records from offset {offset} @ page {page_number}")
        qrs = sdk.query(query, page_number=page_number, page_size=PAGE_SIZE, ttl_minutes=1)
        if qrs.records is None:
            continue
        all_records.extend(qrs.records)
        print(f"Records returned thus far: {len(all_records)}")
        if len(all_records) == count:
            break
    
print(f"Query took {datetime.now() - start_recorded_time} in total to run")
print(f"Total number of records returned: {len(all_records)}")

Querying 100000 records from offset 0 @ page 1
Records returned thus far: 100000
Querying 100000 records from offset 0 @ page 2
Records returned thus far: 200000
Querying 100000 records from offset 0 @ page 3
Records returned thus far: 300000
Querying 100000 records from offset 0 @ page 4
Records returned thus far: 400000
Querying 100000 records from offset 0 @ page 5
Records returned thus far: 500000
Querying 100000 records from offset 0 @ page 6
Records returned thus far: 600000
Querying 100000 records from offset 0 @ page 7
Records returned thus far: 700000
Querying 100000 records from offset 0 @ page 8
Records returned thus far: 800000
Querying 100000 records from offset 0 @ page 9
Records returned thus far: 900000
Querying 100000 records from offset 0 @ page 10
Records returned thus far: 1000000
Querying 100000 records from offset 1000000 @ page 1
Records returned thus far: 1100000
Querying 100000 records from offset 1000000 @ page 2
Records returned thus far: 1193027
Query took 0

In [20]:
all_records[0]

{'block_number': 15439582,
 'block_timestamp': '2022-08-30 09:23:02.000',
 'tx_hash': '0xc0d4536f2ca9a049280086f960f89a43dd42abfda175367756c25e4298de47fc',
 'origin_function_signature': '0x2e95b6c8',
 'origin_from_address': '0x30cb2c51fc4f031fa5f326d334e1f5da00e19ab5',
 'origin_to_address': '0x1111111254fb6c44bac0bed2854e76f90643097d',
 'contract_address': '0x367e2d443988e4b222fbfdafdb35eeb7dda9fbb7',
 'pool_name': 'WETH-DEUS LP',
 'event_name': 'Swap',
 'amount_in': 22.638031861,
 'amount_in_usd': None,
 'amount_out': 0.6135481076,
 'amount_out_usd': None,
 'sender': '0x1111111254fb6c44bac0bed2854e76f90643097d',
 'tx_to': '0x397ff1542f962076d0bfe58ea045ffa2d347aca0',
 'event_index': 71,
 'platform': 'uniswap-v2',
 'token_in': '0xde5ed76e7c05ec5e4572cfc88d1acea165109e44',
 'token_out': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
 'symbol_in': 'DEUS',
 'symbol_out': 'WETH',
 '_log_id': '0xc0d4536f2ca9a049280086f960f89a43dd42abfda175367756c25e4298de47fc-71',
 '_inserted_timestamp': '20